In [6]:
import pandas as pd
import numpy as np
from amuse.units import units

In [5]:
a = [0,1,2,3]
b = [2,3,4,5]
c = [2,5,6,2]

df = pd.DataFrame({'a':a,
                   'b':b,
                   'c':c})
df

,a,b,c
0,0,2,2
1,1,3,5
2,2,4,6
3,3,5,2


In [10]:
end = 10|units.yr
dt = 1|units.yr
times = np.arange(0,end.value_in(units.yr),dt.value_in(units.yr))*1|units.yr
times

quantity<[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] yr>